**Color Mapper**
1. Provides a set of function for color space conversions between sRGB, CIE XYZ, LAB etc.
2. Provides a set of functions to compute the difference between 2 similar colors.


**Useful colormap functions to convert between different color spaces:**
- Convert from sRGB to linear sRGB (vice versa)
- Convert from linear sRGB to CIE XYZ (vice versa)
- Convert from CIE XYZ to CIE LAB (vice versa)
- Convert from CIE XYZ to sRGB


Python colormaps ref:
http://pydoc.net/pwkit/0.8.15/pwkit.colormaps/

In [2]:
import os
import numpy as np
import cv2 
import matplotlib.pyplot as plt

In [11]:
# I don't quite understand where this value comes from, given the various
# Wikipedia values for D65, but this works.
CIELAB_D65 = np.asarray ([0.9505, 1., 1.0890])

X, Y, Z = range(3)
L, A, B = range(3)

# from Moreland:
_linsrgb_to_xyz = np.asarray ([[0.4124, 0.2126, 0.0193],
                               [0.3576, 0.7152, 0.1192],
                               [0.1805, 0.0722, 0.9505]])

# from Wikipedia, SRGB:
_xyz_to_linsrgb = np.asarray ([[3.2406, -0.9689, 0.0557],
                               [-1.5372, 1.8758, -0.2040],
                               [-0.4986, 0.0415, 1.0570]])


def srgb_to_linsrgb (srgb):
    """Convert sRGB values to physically linear ones. The transformation is
    uniform in RGB, so *srgb* can be of any shape.

    *srgb* values should range between 0 and 1, inclusively.

    """
    srgb = srgb/255.0
    gamma = ((srgb + 0.055) / 1.055)**2.4
    scale = srgb / 12.92
    return np.where (srgb > 0.04045, gamma, scale)

def linsrgb_to_srgb (linsrgb):
    """Convert physically linear RGB values into sRGB ones. The transform is
    uniform in the components, so *linsrgb* can be of any shape.

    *linsrgb* values should range between 0 and 1, inclusively.

    """
    # From Wikipedia, but easy analogue to the above.
    gamma = 1.055 * linsrgb**(1./2.4) - 0.055
    scale = linsrgb * 12.92
    linsrgb = np.where (linsrgb > 0.0031308, gamma, scale)
    linsrgb = linsrgb * 255.0
    return linsrgb


def linsrgb_to_xyz (linsrgb):
    """Convert linearized sRGB values (cf srgb_to_linsrgb) to CIE XYZ values.

    *linsrgb* should be of shape (*, 3). Values should range between 0 and 1
    inclusively. Return value will be of same shape.

    Returned XYZ values range between [0, 0, 0] and [0.9505, 1., 1.089].

    """
    return np.dot (linsrgb, _linsrgb_to_xyz)


def xyz_to_linsrgb(xyz): 
    """Convert CIE XYZ values to linearized sRGB values (cf srgb_to_linsrgb).

    *xyz* should be of shape (*, 3)

    Return value will be of same shape.

    """
    return np.dot(xyz,_xyz_to_linsrgb)

def xyz_to_cielab(xyz,refwhite=CIELAB_D65):
    """Convert CIE xyz color values to CIE L*a*b.
    *xyz* should be of shape (*,3). *refwhite* is the reference white value of,
    shape(3,)
    
    Return value will have same shape as *xyz*, but be in CIE L*a*b
    coordinates.
    """
    norm = xyz/refwhite
    pow = norm**0.333333333333333
    scale = 7.787037 * norm + 16./116
    mapped = np.where(norm >  0.008856, pow, scale)
    
    cielab = np.empty_like(xyz)
    cielab[...,L] = 116 * mapped[...,Y] - 16
    cielab[...,A] = 500 * (mapped[...,X] - mapped[...,Y])
    cielab[...,B] = 200 * (mapped[...,Y] - mapped[...,Z])
    
    return cielab


    
def cielab_to_xyz(cielab,refwhite=CIELAB_D65):
    """Convert CIE L*a*b* color values to CIE XYZ,

    *cielab* should be of shape (*, 3). *refwhite* is the reference white
    value in the L*a*b* color space, of shape (3, ).

    Return value has same shape as *cielab*

    """
    def func (t):
        pow = t**3
        scale = 0.128419 * t -  0.0177129
        return np.where(t > 0.206897,pow,scale)
    
    xyz = np.empty_like(cielab)
    lscale = 1./116 * (cielab[...,L] + 16)
    xyz[...,X] = func(lscale + 0.002 * cielab[...,A])
    xyz[...,Y] = func(lscale)
    xyz[...,Z] = func(lscale - 0.005 * cielab[...,B])
    xyz *= refwhite
    return xyz


def srgb_to_cielab(srgb):
    linsrgb = srgb_to_linsrgb(srgb)
    xyz = linsrgb_to_xyz(linsrgb)
    cielab = xyz_to_cielab(xyz)
    return cielab
    
    
def test_color_conversions():
    testRGB = np.array([118,118,118])
    print("The value of 18% gray - sRGB: ", testRGB)
    
    testsRGB_linsRGB = srgb_to_linsrgb(testRGB)
    print("The value of 18% gray, sRGB - linear RGB: ", testsRGB_linsRGB)
    
    testlinsRGB_sRGB = linsrgb_to_srgb(testsRGB_linsRGB)
    print("The value of 18% gray, linear RGB - sRGB: ", testlinsRGB_sRGB)
    
    testlinsRGB_XYZ = linsrgb_to_xyz(testsRGB_linsRGB)
    print("The value of 18% gray - linear sRGB to CIE XYZ: ", testlinsRGB_XYZ)
    
    testlinsRGB_XYZ_LAB = xyz_to_cielab(testlinsRGB_XYZ)
    print("The value of 18% gray - CIE XYZ to CIE LAB: ", testlinsRGB_XYZ_LAB)
    
    testLAB_XYZ = cielab_to_xyz(testlinsRGB_XYZ_LAB)
    print("The value of 18% gray - CIE LAB to CIE XYZ: ", testLAB_XYZ)
    
    testXYZ_linsRGB = xyz_to_linsrgb(testLAB_XYZ)
    print("The value of 18% gray - CIE XYZ to linear RGB: ", testXYZ_linsRGB)
    
    testlinsRGB_sRGB = linsrgb_to_srgb(testXYZ_linsRGB)
    print("The value of 18% gray - linear RGB to sRGB: ", np.round(testlinsRGB_sRGB))

In [12]:
test_color_conversions()

The value of 18% gray - sRGB:  [118 118 118]
The value of 18% gray, sRGB - linear RGB:  [0.18116424 0.18116424 0.18116424]
The value of 18% gray, linear RGB - sRGB:  [118. 118. 118.]
The value of 18% gray - linear sRGB to CIE XYZ:  [0.17219661 0.18116424 0.19728786]
The value of 18% gray - CIE XYZ to CIE LAB:  [49.63701437  0.          0.        ]
The value of 18% gray - CIE LAB to CIE XYZ:  [0.17219661 0.18116424 0.19728786]
The value of 18% gray - CIE XYZ to linear RGB:  [0.18116694 0.18117404 0.18116712]
The value of 18% gray - linear RGB to sRGB:  [118. 118. 118.]


In [13]:
srgb2cielab = np.array([255,255,255])
srgb2cielab = srgb_to_cielab(srgb2cielab)
print(srgb2cielab)

[100.   0.   0.]
